# PyKS2

ZC. Fang (Copyright &copy; 2018)

一个读取日本共和电业（Kyowa）KS2数据文件的python库。提供读取、转存.mat格式和基本的绘图功能。

依赖python包：

+ numpy >= 1.13
+ scipy >= 0.19
+ pandas >= 0.20
+ matplotlib >= 2.0

## 导入PyKS2及相关依赖包

In [ ]:
from pyks2 import *

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# 图片行内显示
%matplotlib inline

In [ ]:
print("Numpy version:", np.__version__)
print("Scipy version:", sp.__version__)
print("Pandas version:", pd.__version__)
print("Matplotlib version:", mpl.__version__)

## 载入KS2文件

In [ ]:
ks2 = KS2("example.ks2")

## 转存格式

In [ ]:
# 指定导出文件名savename
# 后缀名.mat
ks2.save(savename='example.mat')

## 绘图

In [ ]:
# select channel
# 0 ~ chN-1 (type: list)
# None -> all channels
# ch = None
ch = [1]

# time range (None or [t1, t2])
# tRange = None
tRange = [1, 5]

if ch is None:
    ch = range(ks2.chN)

# select downsampled sampling frequency
fsDS = 250

# down sample the data for plotting
di = round(max(1, ks2.fs / fsDS))
if tRange is None:
    rawDS = ks2.raw[::di, :]
else:
    rawDS = ks2.raw[round(tRange[0] * ks2.fs):round(tRange[1] * ks2.fs):di, :]
    
if len(ch) == 1:
    dataDS = np.array(list(map(
        lambda i: ks2.coefA[i] * rawDS[:, i] + ks2.coefB[i], ch))).reshape((-1, 1))
else:
    dataDS = np.array(
        list(map(lambda i: ks2.coefA[i] * rawDS[:, i] + ks2.coefB[i], ch))).T

if tRange is None:
    t = np.arange(dataDS.shape[0]) / (ks2.fs / di)
else:
    t = tRange[0] + np.arange(dataDS.shape[0]) / (ks2.fs / di)

chNames = list(map(lambda i: ks2.chName[i], ch))
dataframe = pd.DataFrame(dataDS, index=t, columns=chNames)
ax = dataframe.plot(figsize=(12, 10), linewidth=1)
# ax.set_xlim([6, 6.5])
ax.set_xlabel(r'$t$ (s)')
ax.legend(loc=1, fontsize='small')
